In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# from ipynb.fs.full.Preprocessing import clean_data
# %load_ext tensorboard

# df = pd.read_csv('./Data/songs_cleaned.csv', index_col='Unnamed: 0')
# print(df.shape)
# df.describe()

In [16]:
df = pd.read_csv('Data/songs_cleaned.csv', index_col='Unnamed: 0')

In [17]:
# Simple autoencoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

h1 = Dense(32, activation = 'relu')(input_song)

encoded = Dense(encoded_dim, activation = 'relu')(h1)

dh1 = Dense(32, activation='sigmoid')(encoded)

decoded = Dense(df.shape[1], activation='sigmoid')(dh1)

autoencoder = Model(input_song, decoded)

encoder = Model(input_song, encoded)

autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [18]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

2021-08-24 14:56:32.531680: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-24 14:56:32.531693: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-24 14:56:32.532346: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/100
  132/14659 [..............................] - ETA: 21s - loss: 0.6295

2021-08-24 14:56:33.528533: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-24 14:56:33.528551: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-24 14:56:33.583476: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-24 14:56:33.585707: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-24 14:56:33.587790: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/SimpleAutoencoder-20210824-145632/train/plugins/profile/2021_08_24_14_56_33
2021-08-24 14:56:33.591817: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/SimpleAutoencoder-20210824-145632/train/plugins/profile/2021_08_24_14_56_33/Petrs-MBP.trace.json.gz
2021-08-24 14:56:33.598667: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/SimpleAu

14659/14659 [==============================] - 12s 778us/step - loss: 0.3391 - val_loss: 0.3306
Epoch 2/100
14659/14659 [==============================] - 11s 766us/step - loss: 0.3000 - val_loss: 0.3204
Epoch 3/100
14659/14659 [==============================] - 11s 746us/step - loss: 0.2964 - val_loss: 0.3192
Epoch 4/100
14659/14659 [==============================] - 11s 755us/step - loss: 0.2958 - val_loss: 0.3188
Epoch 5/100
14659/14659 [==============================] - 10s 707us/step - loss: 0.2955 - val_loss: 0.3186
Epoch 6/100
14659/14659 [==============================] - 11s 771us/step - loss: 0.2952 - val_loss: 0.3183


### Shallow NN Summary:
0 hidden layers: Val Loss: 0.3160
1 hidden layer: 

In [19]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

In [20]:
print(embeddings_df.shape)
embeddings_df

(586344, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,5.811045,3.451632,3.994831,5.511910,4.919834,3.044781,3.112660,3.354124,6.345873,4.466634,4.130972,6.135262,3.275150,3.002502,5.727569,0.904308
1,3.988864,5.090059,4.005454,5.161570,4.790668,4.588681,0.417547,2.158737,4.630351,4.821034,3.667093,5.070941,2.295691,3.771557,6.311641,1.645125
2,3.000980,3.536582,5.520813,2.077629,3.516592,2.760815,1.707682,4.890491,2.623950,2.032806,4.277054,3.794971,3.801049,4.777055,2.814221,2.757380
3,6.570363,0.069064,4.043518,3.233943,5.879925,1.524485,3.567957,4.287617,5.682148,5.047094,4.310845,4.858367,3.951431,3.438916,4.727888,2.798956
4,3.150914,1.749139,5.063042,1.830458,2.902534,2.567786,1.168590,4.127640,2.712111,2.530576,2.210352,3.490625,3.278237,1.842046,2.327283,2.806181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,2.292302,1.697363,2.254984,1.255893,2.888316,3.004323,3.632368,4.397654,2.434651,3.574094,3.051071,3.136074,1.637554,1.691380,2.886303,2.995786
586668,2.353943,2.786595,2.182580,3.360007,4.913273,4.613508,4.785804,4.559988,3.171568,3.405686,4.994813,2.730458,2.534873,2.783816,4.489384,3.680459
586669,2.196050,0.099318,1.654299,0.777018,3.038073,2.852736,2.065424,4.673674,1.229042,3.118731,2.641318,1.487261,1.150925,1.069228,3.246775,2.139384
586670,1.615683,1.668802,1.522438,3.189543,4.621633,3.847653,3.984465,4.301698,1.959500,1.126728,2.328117,4.421263,2.647792,2.009002,4.350246,3.955700


In [21]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                 radius=1.0)

In [22]:
len(encoded_songs[0])

16

In [ ]:
# Saving the encoded_songs embedding as a df and the Nearest Neighbours model for use in our app
import pickle

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

# Saving the embeddings to the disk
embeddings_df.to_csv('./saved_models/embeddings_df_001.csv')

# save the model to disk
filename = './saved_models/nearestneigh_001.sav'
pickle.dump(nn, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [16]:
embeddings_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,3.470329,1.184762,-0.0,3.144354,1.041142,2.506661,2.963388,1.308488,1.876889,1.609339,5.813118,4.160755,2.801373,6.369508,4.479137,2.048349
1,2.950846,0.692970,-0.0,1.260800,2.372062,1.424065,1.766102,3.973808,1.751205,1.503640,4.932892,4.102685,1.840484,4.968472,4.601698,2.108422
2,4.472703,6.916587,-0.0,3.051311,4.255427,4.078246,6.610500,5.424572,5.226394,2.885791,4.807835,8.209723,6.890685,2.668237,4.231436,3.645994
3,0.533942,1.481426,-0.0,4.247745,1.795213,3.724570,6.466702,2.883682,2.688429,2.568929,3.822564,2.750004,4.210648,3.195352,3.263384,3.226213
4,2.028276,4.151319,-0.0,2.583655,2.820859,2.327025,2.896990,3.921444,3.261746,1.292208,6.060862,3.102732,4.927852,3.230224,1.068353,2.544864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,4.507507,4.525337,-0.0,1.797992,4.408144,3.942313,3.207572,3.014376,2.259826,3.485083,7.321132,4.829626,4.268849,4.802366,3.543452,2.489765
586668,4.967984,3.997606,-0.0,2.682946,5.384268,4.684574,2.361519,3.863641,2.298209,4.028028,7.587534,5.696170,5.347732,3.508698,5.025582,3.510531
586669,2.513383,4.620409,-0.0,3.026764,4.275283,6.560318,5.074978,4.096501,2.595194,1.959881,3.780759,4.040783,5.324980,2.569675,2.156150,2.363571
586670,4.446574,4.300193,-0.0,4.233162,4.264293,5.235272,1.241963,4.270548,4.267136,2.311171,6.389156,5.381336,5.270690,4.331300,3.387321,5.655649


In [23]:
# Testing our model with multiple songs
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes


DeepSpot's recommendation system:


Top 10 recommendations for California Love by 175588    ['2Pac']
Name: artists, dtype: object:

Lisboa Antigua by ['Jorge Sepúlveda'] with a score of 0.0

URL: https://open.spotify.com/track/2TE7NXdEmPnQxi69tAaECa

Hyväillä ja hoivata by ['Frederik'] with a score of 1.0386382294544085

URL: https://open.spotify.com/track/5yV3xS10IBHVkupMLz0ppe

Fade to Black - Live / Seattle '89 by ['Metallica'] with a score of 1.2134997379075512

URL: https://open.spotify.com/track/46GRDe7B3ptpFE7brebD69

Neviem byt sam by ['Elán'] with a score of 1.261382514555003

URL: https://open.spotify.com/track/1l2bM511Nfmoc55dsnCeUp

Heavy Metal: The Black and Silver by ['Blue Öyster Cult'] with a score of 1.2747358413875256

URL: https://open.spotify.com/track/1uUCpOX86yRGShambUp93O

I Remember Jim by ['Anita Kerr Singers'] with a score of 1.2759895467963422

URL: https://open.spotify.com/track/03p1DF2p7e0kRw4UPWDZri

El Sonido De La Ciudad - Remastered by ['Miguel Ríos'] w

In [ ]:
# Deep variational encoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

encoded_h1 = Dense(64, activation = 'relu')(input_song)
encoded_h2 = Dense(32, activation = 'relu')(encoded_h1)
encoded = Dense(encoded_dim, activation = 'relu')(encoded_h2)

decoded_h1 = Dense(32, activation='relu')(encoded)
decoded_h2 = Dense(64, activation='relu')(decoded_h1)
decoded = Dense(df.shape[1], activation='sigmoid')(decoded_h2)

deep_autoencoder = Model(input_song, decoded) # techincally autoencoder_and_decoder

deep_encoder = Model(input_song, encoded) # Half of the autoencoder

deep_autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

deep_autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Deep NN Summary
Val loss = 0.2960

In [32]:
df.dtypes

popularity          float64
duration_ms         float64
explicit            float64
danceability        float64
energy              float64
key_1                 int64
key_2                 int64
key_3                 int64
key_4                 int64
key_5                 int64
key_6                 int64
key_7                 int64
key_8                 int64
key_9                 int64
key_10                int64
key_11                int64
key_12                int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature      float64
dtype: object

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

In [ ]:
encoded_songs.shape

In [ ]:
# # Testing
# raw_df = pd.read_csv('./Data/tracks.csv')

# test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
#               'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# # Loop over all our test songs and get the recommendations
# recommendations = {}

# print('DeepSpot\'s recommendation system:\n\n')

# for name, id in test_songs.items():
#     query = raw_df.index[raw_df['id'] == id][0]
#     test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
#     score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
#     k_closest_songs_indexes = close_image_idx[0][1:]
    
#     print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
#     for i in range(len(k_closest_songs_indexes)):
#         song = raw_df.iloc[k_closest_songs_indexes[i], :]
#         print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
#         print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
#     print('___________________________________________________________________________________\n')
    
#     recommendations[name] = k_closest_songs_indexes



# Training a Variational Autoencoder

In [24]:
# Variational Autoencoder
from keras import layers

class Sampling(layers.Layer):
    """
    Uses (z_mean, z_log_var) to sample a z vector from the latent N-dimensional Normal distribution
    i.e. Z is the vector encoding a digit.
    """
    def call(self, inputs):
        
        # recall from the video that z_mean and z_log_var are vectors 
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        
        # sample from an N-dimensional normal distribution
        # epsilon is given shape (batch, dim) because we are adding it to z_mean which has shape (batch, dim)
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        # output of tf.exp() is a vector 
        simga = tf.exp(0.5 * z_log_var)
        
        # this is our hideen latent vector made up of a mean and variance vector 
        # variance vector is scaled by epsilon, which is sampled from a normal distribtuion 
        # the the video guy said "stocastic" in reference to epsilon, he meant random 
        Z = z_mean + simga * epsilon
        
        # return hidden latent vector 
        return Z

In [96]:
# recall from the video that the more dimensions that our latent vector has
# the better the results of our model 
latent_dim = 8

# shape of our input data
# we are creating our input layer using Keras's Input() class
# the only thing that input layers really do is define the dimensionality of the input data for the model
encoder_inputs = Input(shape=(df.shape[1], ))

# these are the hidden layers

# pass data vector into FCFF layer 
encoded = Dense(encoded_dim, activation = 'relu')(encoder_inputs)

# recall that ordinarly the output of the last encoding layer is the latent vector 
# but here we are creating two output layers for our encoder - one for the mean and one for the log variance 
# returns a 2-dim mean vector
z_mean = Dense(latent_dim, name="z_mean")(encoded)
# returns a 2-dim log variance vecotr 
z_log_var = Dense(latent_dim, name="z_log_var")(encoded)
# pass mean and variance vector into Sampling class in order to create the Z vector,  Z = mean + var * epsilon
z = Sampling()([z_mean, z_log_var])

# ok, now let's put it all together 
# this is our encoder model 
# inputs are the original images
# outputs are the Z vectors: mean, log variance, and the complete Z, i.e. vector Z = mean + var * epsilon
encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 26)]         0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 16)           432         input_13[0][0]                   
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 8)            136         dense_22[0][0]                   
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 8)            136         dense_22[0][0]                   
____________________________________________________________________________________________

In [97]:
# the input layer to our decoder has the same dimensionality as the latent vector
# because the latent vector is the input to the decoder model
latent_inputs = Input(shape=(latent_dim,))

# these are the hidden layers of our decoder 
# the data at this point is in a vector, the Z latent vector 

# this is the final layer in out decoder
# therefore this layer outputs the reconstruction of the original image 
decoder_outputs = Dense(df.shape[1], activation='sigmoid')(latent_inputs)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_23 (Dense)             (None, 26)                234       
Total params: 234
Trainable params: 234
Non-trainable params: 0
_________________________________________________________________


In [33]:
from tensorflow import keras

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        """
        This class build a Variational Auto-Encoder. It accepts an Encoder and Decoder model as input. 
        
        Note
        ----
        This VAE class is inheriting Keras's Model API so that it can use the Model class methods 

        """
        # how python 3 handels inheritance 
        super(VAE, self).__init__(**kwargs)
        # set encoder model as class attribute
        self.encoder = encoder
        # set decoder model as class attribute 
        self.decoder = decoder
        # set mean function as class attribute - this calculates the total loss
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        # set mean function as class attribute - this calculates the reconstruction loss
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        # set mean function as class attribute - this calculates the kl loss
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        """
        Returns all loses in a list
        """
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        """
        Training our model via gradient descent and back-propagation 
        """
        
        # we used tf.GradientTape() in Sprint 2 Module 2 to run Gradient Descent from scratch 
        with tf.GradientTape() as tape:
            # pass input data into encoder model 
            # output of encoder model is the hidden state distribution parameters and hidden state vector 
            z_mean, z_log_var, z = self.encoder(data)
            
            # pass hidden state vector into decoder model 
            reconstruction = self.decoder(z)
            
            # calculate the reconstruction loss 
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction)#, axis=(1, 2)
                )
            )
            
            # calculate the kl loss
            #                (1 + z_simga   - (z_mean)^2        - e^(z_simga) ) 
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            # recall that we used tf.reduce_sum() in Sprint 2 Module 4 assignment 
            # it takes the sum of the vector components 
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            # calculate the total loss by adding the two loss components 
            total_loss = reconstruction_loss + kl_loss
            
        # now that we have calculated the loss function, we can perform Gradient Descent
        # we are passing in the loss function and the weights that we want to update via Gradeint Descent 
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # log the total loss
        self.total_loss_tracker.update_state(total_loss)
        
        # log the reconstruction loss 
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        
        # log the kl loss 
        self.kl_loss_tracker.update_state(kl_loss)
        
        # return all the losses in a dictionary 
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [98]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, restore_best_weights=True)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"VAE-{now}")
tensorboard = TensorBoard(log_dir=logdir)

# train the model weights 
vae.fit(df, epochs=100,
        callbacks=[stop, tensorboard])

2021-08-25 10:37:44.631468: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 10:37:44.631485: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-25 10:37:44.631659: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/100
  125/18324 [..............................] - ETA: 30s - loss: 25.0390 - reconstruction_loss: 23.1630 - kl_loss: 1.2593

2021-08-25 10:37:45.749862: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 10:37:45.749879: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-25 10:37:45.810550: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-25 10:37:45.812944: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-25 10:37:45.815861: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/VAE-20210825-103744/train/plugins/profile/2021_08_25_10_37_45
2021-08-25 10:37:45.817346: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/VAE-20210825-103744/train/plugins/profile/2021_08_25_10_37_45/Petrs-MBP.trace.json.gz
2021-08-25 10:37:45.821984: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/VAE-20210825-103744/train/plugins/pr

18324/18324 [==============================] - 16s 859us/step - loss: 17.4896 - reconstruction_loss: 14.8143 - kl_loss: 1.56901s - loss: 17.6129 - reconstruc
Epoch 2/100
18324/18324 [==============================] - 14s 745us/step - loss: 15.6907 - reconstruction_loss: 13.9358 - kl_loss: 1.7640
Epoch 3/100
18324/18324 [==============================] - 14s 750us/step - loss: 15.6536 - reconstruction_loss: 13.8345 - kl_loss: 1.8127
Epoch 4/100
18324/18324 [==============================] - 14s 748us/step - loss: 15.6581 - reconstruction_loss: 13.7940 - kl_loss: 1.8307
Epoch 5/100
18324/18324 [==============================] - 14s 758us/step - loss: 15.6480 - reconstruction_loss: 13.7708 - kl_loss: 1.8459
Epoch 6/100
18324/18324 [==============================] - 14s 759us/step - loss: 15.5487 - reconstruction_loss: 13.7483 - kl_loss: 1.8573
Epoch 7/100
18324/18324 [==============================] - 14s 747us/step - loss: 15.6579 - reconstruction_loss: 13.7274 - kl_loss: 1.8741
Epoch 8/

# VAE Summary:
loss: 15.5321 - reconstruction_loss: 13.6563 - kl_loss: 1.9151

In [51]:
# Encoding our songs
z_mean, z_log_var, z = vae.encoder.predict(df)

embeddings_df = pd.DataFrame(z, index=df.index)

# fitting KNN

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(z)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                 radius=1.0)

In [66]:
# Testing the VAE - It seems to work!
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(z[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes




DeepSpot's recommendation system:


Top 10 recommendations for California Love by 175588    ['2Pac']
Name: artists, dtype: object:

Torpedo Blu by ['Giorgio Gaber'] with a score of 2.0387225534784523

URL: https://open.spotify.com/track/2hWsjxSO7bIKHzCBQMmmA0

I Am a Rocker - Live at Abbey Road; 2005 Remaster by ['Chas & Dave'] with a score of 2.042183201276265

URL: https://open.spotify.com/track/1rNfjUa8wtQNeg5CpnZrpw

September '99 - Phats & Small Remix by ['Earth, Wind & Fire', 'Phats & Small'] with a score of 2.0909346043421233

URL: https://open.spotify.com/track/6Z6my7Cpi5BongU2fsY96l

Mera Yaar by ['Shankar-Ehsaan-Loy', 'Javed Bashir'] with a score of 2.0985039762456625

URL: https://open.spotify.com/track/2NECp5Tvb5Lm6TbJf0Y8xI

Tracks of My Tears by ['Smokey Robinson'] with a score of 2.1003578809715298

URL: https://open.spotify.com/track/5QizJ1IfhrJemVys14KlJF

It Don't Matter to Me by ['Bread'] with a score of 2.104974467345315

URL: https://open.spotify.com/track/5s8lZd5T

# Hyperparameter tuning the VAE

In [82]:
latent_dim = 8

encoder_inputs = Input(shape=(df.shape[1], ))

h1 = Dense(16, activation = 'relu')(encoder_inputs)
encoded = Dense(encoded_dim, activation = 'relu')(h1)

z_mean = Dense(latent_dim, name="z_mean")(encoded)

z_log_var = Dense(latent_dim, name="z_log_var")(encoded)

z = Sampling()([z_mean, z_log_var])

encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 26)]         0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 16)           432         input_11[0][0]                   
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 16)           272         dense_18[0][0]                   
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 8)            136         dense_19[0][0]                   
____________________________________________________________________________________________

In [83]:
latent_inputs = Input(shape=(latent_dim,))

dh1 = Dense(16, activation='relu')(latent_inputs)

decoder_outputs = Dense(df.shape[1], activation='sigmoid')(dh1)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_20 (Dense)             (None, 16)                144       
_________________________________________________________________
dense_21 (Dense)             (None, 26)                442       
Total params: 586
Trainable params: 586
Non-trainable params: 0
_________________________________________________________________


In [84]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, restore_best_weights=True)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"DeepVAE-{now}")
tensorboard = TensorBoard(log_dir=logdir)

# train the model weights 
vae.fit(df, epochs=100,
        callbacks=[stop, tensorboard])

2021-08-25 10:26:16.779240: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 10:26:16.779260: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-25 10:26:16.779510: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/100
  105/18324 [..............................] - ETA: 41s - loss: 25.9393 - reconstruction_loss: 23.2566 - kl_loss: 0.9896 

2021-08-25 10:26:18.450155: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-25 10:26:18.450169: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-25 10:26:18.522889: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-25 10:26:18.525891: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-25 10:26:18.529263: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/DeepVAE-20210825-102616/train/plugins/profile/2021_08_25_10_26_18
2021-08-25 10:26:18.531174: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/DeepVAE-20210825-102616/train/plugins/profile/2021_08_25_10_26_18/Petrs-MBP.trace.json.gz
2021-08-25 10:26:18.537788: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/DeepVAE-20210825-102616/trai

18324/18324 [==============================] - 17s 880us/step - loss: 17.0531 - reconstruction_loss: 15.2413 - kl_loss: 1.0129
Epoch 2/100
18324/18324 [==============================] - 15s 833us/step - loss: 15.6979 - reconstruction_loss: 14.2567 - kl_loss: 1.4092
Epoch 3/100
18324/18324 [==============================] - 16s 849us/step - loss: 15.6211 - reconstruction_loss: 14.1369 - kl_loss: 1.4779
Epoch 4/100
18324/18324 [==============================] - 16s 893us/step - loss: 15.5848 - reconstruction_loss: 14.0539 - kl_loss: 1.5281
Epoch 5/100
18324/18324 [==============================] - 18s 964us/step - loss: 15.5693 - reconstruction_loss: 14.0180 - kl_loss: 1.5495
Epoch 6/100
18324/18324 [==============================] - 16s 898us/step - loss: 15.5970 - reconstruction_loss: 14.0010 - kl_loss: 1.5576
Epoch 7/100
18324/18324 [==============================] - 16s 866us/step - loss: 15.4935 - reconstruction_loss: 13.9793 - kl_loss: 1.5698
Epoch 8/100
18324/18324 [==============

Results deep VAE v1 (latent_dim = 16, h1 = 32): 
loss: 15.9362 - reconstruction_loss: 14.5233 - kl_loss: 1.4098

**Results deep VAE v2 (latent_dim = 8, h1 = 16):**
Epoch 21/100
loss: 15.4746 - reconstruction_loss: 13.8923 - kl_loss: 1.6215

Results regular VAE v1 (latent_dim = 16): 
loss: 15.5321 - reconstruction_loss: 13.6563 - kl_loss: 1.9151

Results regular VAE v1 (latent_dim = 8):
Epoch 12/100
loss: 15.5430 - reconstruction_loss: 13.7091 - kl_loss: 1.8810

In [95]:
# Testing results
# Encoding our songs
z_mean, z_log_var, z = vae.encoder.predict(df)

embeddings_df = pd.DataFrame(z, index=df.index)

# fitting KNN

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(z)

# Testing the VAE v2
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(z[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes

embeddings_df.to_csv('./Data/vae_embeddings.csv')



DeepSpot's recommendation system:


Top 10 recommendations for California Love by 175588    ['2Pac']
Name: artists, dtype: object:

Psycho (feat. Ty Dolla $ign) by ['Post Malone', 'Ty Dolla $ign'] with a score of 0.5826315098088156

URL: https://open.spotify.com/track/3swc6WTsr7rl9DqQKQA55C

Han Mikkel by ['Unit Five'] with a score of 0.6039448575863513

URL: https://open.spotify.com/track/0SSDgzZ20kSjzRMsx4CXTo

Verdi: Rigoletto, Act 2: "Tutte le feste al tempio" (Rigoletto, Gilda) by ['Giuseppe Verdi', 'Tito Gobbi', 'Orchestra Del Teatro Alla Scala, Milano', 'Tullio Serafin'] with a score of 0.6166760292215672

URL: https://open.spotify.com/track/1rsvdwKNP7r5cRjPf74Yw9

สัญญาเจ้าลืม by ['หนู มิเตอร์'] with a score of 0.6176218466138796

URL: https://open.spotify.com/track/1IzLBR8lmdKmmagDAzolA6

Pissikset by ['Lissut'] with a score of 0.6213709595482744

URL: https://open.spotify.com/track/2R22hS7NsTueQrG7z4Fq5d

Gaucho by ['Steely Dan'] with a score of 0.6272687563675855

URL: https

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
autoencoder.save('saved_model/vae_002')
encoder.save('saved_model/encoder_002')